## Artificial Neural Network and TensorFlow
[Artificial Neural Network](https://en.wikipedia.org/wiki/Artificial_neural_network) (ANNs) are a family of models inspired by biological neural networks which are used to estimate or approximate functions that can depend on a large number of inputs and are generally unknown.  

### Biological Neuron
* There are about one hundred billion neurons in human brain.
* Brain is an extremeley interconnected networks of neurons. 
* A neuron collects inputs using a structure called dendrites, the neuron effectively sums all of these inputs from the dendrites and if the resulting value is greater than it's firing threshold, the neuron fires.
* When the neuron fires it sends an electrical impulse through the neuron's axon to it's boutons. These boutons can then be networked to thousands of other neurons via connections called synapses.
<img src="images/biological_neuron.png" width=50%>  

### Perceptron (or Artificial Neuron)
A typical perceptron will have many inputs and these inputs are all individually weighted. The perceptron weights can either amplify or deamplify the original input signal. For example, if the input is 1 and the input's weight is 0.2 the input will be decreased to 0.2. These weighted signals are then added together and passed into the activation function. The activation function is used to convert the input into a more useful output. There are many different types of activation function but one of the simplest would be step function. A step function will typically output a 1 if the input is higher than a certain threshold, otherwise it's output will be 0.  

**Two step process** 
1. Calculate the weighted sum  
2. Use the activation function to decide the output of of the neuron  
<img src="images/perceptron.png" width=50%>

In [ ]:
import numpy as np

def step_activation_function(val, threshold):
    if val>threshold:
        return 1
    else:
        return 0
    
def weighted_sum(inp, wt):
    temp_sum = 0
    for i in range(len(inp)):
        temp_sum += inp[i]*wt[i]
    return temp_sum
    

In [ ]:
#### OR Gate
# Input_1 Input_2 Output
#    0       0      0
#    0       1      1
#    1       0      1
#    1       1      1

inp = [1, 1]
wt = [2.0, 2.0]
threshold = 1.0

## Step 1: Calculate the weighted sum
wt_sum = weighted_sum(inp, wt)

## Step 2: Use the activation function to decide the output of of the neuron
out = step_activation_function(wt_sum, threshold)
print out


### Neural Network
<img src="images/neural_network.png" width=40%>

#### Feed forward Neural Network
A [feedforward neural network](https://en.wikipedia.org/wiki/Feedforward_neural_network) is an artificial neural network wherein connections between the units do not form a cycle.  
* Input Layer: Nodes of the network that accept input values. In the network below, **nodes 1, 2 and 3** are **input nodes**. They do not compute anything, but simply pass the values to the processing nodes in the next layers.
* Ouput Layer: These nodes provide us with the output. **Nodes (6 and 7)** are associated with the **output variables**.
* Hidden Layer: If neural network was a black box then thesea re the layers that are not visible. They compute values depending on the weight values between interconnections. **Nodes (4 and 5)** are **hidden nodes**.

<img src="images/feed_forward.png" width=40%>

#### Exercise:
<img src="images/feed_forward_example.png" width=45% align="middle">

### Training the network (Learning the weights)  
1. Randomly initialize all the weights ($w_{ij}$)
2. Repeat:  
  * Feed the network with an input x from one of the examples in the training set  
  * Compute the network’s output f(x)  
  * Change the weights $w_{ij}$ of the nodes  
3. Until the error is small
### Back propagation algorithm (How to update the weights)
<img src="images/backpropagation.png" width=50%>
#### Further readings on backpropagation
[Detailed step by step backpropagation explained](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/)

#### Preparing the data sets

In [2]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [4]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [5]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [6]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

Epoch: 0001 cost= 176.292299819
Epoch: 0002 cost= 41.740571508
Epoch: 0003 cost= 26.303413380
Epoch: 0004 cost= 18.256388527
Epoch: 0005 cost= 13.365855589
Epoch: 0006 cost= 10.084975574
Epoch: 0007 cost= 7.499880418
Epoch: 0008 cost= 5.667201919
Epoch: 0009 cost= 4.265265011
Epoch: 0010 cost= 3.153722109
Epoch: 0011 cost= 2.495421207
Epoch: 0012 cost= 1.858476185
Epoch: 0013 cost= 1.339489388
Epoch: 0014 cost= 1.003348840
Epoch: 0015 cost= 0.835658200
Optimization Finished!
Accuracy: 0.9489


In [19]:
import tensorflow.contrib.learn as skflow
from sklearn import metrics

mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

classifier = skflow.TensorFlowDNNClassifier(hidden_units=[256, 256], n_classes=10)
classifier.fit(mnist.train.images, mnist.train.labels.astype(int))
score = metrics.accuracy_score(mnist.test.labels.astype(int), classifier.predict(mnist.test.images))
print("Accuracy: %f" % score)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.


Accuracy: 0.903000


#### Recurrent Neural Networks
A recurrent neural network (RNN) is a class of artificial neural network where connections between units form a directed cycle. This creates an internal state of the network which allows it to exhibit dynamic temporal behavior. Unlike feedforward neural networks, RNNs can use their internal memory to process arbitrary sequences of inputs. This makes them applicable to tasks such as unsegmented connected handwriting recognition or speech recognition.  

##### [LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) 


In [ ]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})

#### RNN implementation with skflow (Need to update)

In [30]:
import tensorflow.contrib.learn as skflow

def mnist_rnn_input_op_fn(x):
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, 28])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, 28, x)
    return x

classifier = skflow.TensorFlowRNNClassifier(rnn_size=28, cell_type='rnn', n_classes=10, input_op_fn=mnist_rnn_input_op_fn)

#classifier = skflow.TensorFlowRNNClassifier(rnn_size=28, 
#    n_classes=10, cell_type='rnn', input_op_fn=mnist_rnn_input_op_fn,
#    num_layers=1, bidirectional=False, sequence_length=None,
#    steps=1000, optimizer='Adam', learning_rate=0.01, continue_training=True)
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
train_data = mnist.train.images.reshape((55000, 28, 28))
classifier.fit(train_data, mnist.train.labels)
print mnist.test.images.shape
pred = classifier.predict(mnist.test.images.reshape((10000, 28, 28)))
score = metrics.accuracy_score(mnist.test.labels.astype(int), pred)
print score
#print mnist.test.labels[0]
#correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(mnist.test.labels, 1))
#print correct_prediction
#correct_prediction = tf.equal(tf.argmax(classifier.predict(mnist.test.images).reshape(10000, 28, 28), 1), tf.argmax(mnist.test.labels, 1))
    # Calculate accuracy
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})


(55000, 784)
(55000,)
(10000, 784)
0.3952


#### Convolutional Neural Networks

#### Deep Neural Networks